# Load Database Tables into Python

In [1]:
# Import Dependencies
import sqlalchemy
from sqlalchemy import create_engine
from config import db_password
import pandas as pd

In [2]:
# Import dependencies
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
from keras.utils import to_categorical
from keras.layers.advanced_activations import LeakyReLU

import matplotlib.pyplot as plt
import numpy as np


Using TensorFlow backend.


In [3]:
# read data from Postgres Database
engine = create_engine('postgresql://postgres:'+ db_password +'@localhost:5432/AI_Music_DB')

notes_df = pd.read_sql_table('Notes_Spectrogram_Table',engine)
# notes_df

In [4]:
# Convert Spectrograms from list to ndarray
notes_df['Spectrogram'] = notes_df['Spectrogram'].apply(lambda x: np.array(x))

type(notes_df['Spectrogram'].iloc[0])

numpy.ndarray

# MODEL - Neural Network 

In [5]:
# Create train and test data sets
X_series = notes_df["Spectrogram"]
y = notes_df["Note"]

In [6]:
# Parameters
channels = 1 # number of audio channels
spectrogram_shape = X_series[1].shape + (channels,)
batch = spectrogram_shape[1]

In [7]:
# Reshape X into size of spectrogram and convert to ndarray
X = np.array([i.reshape( (spectrogram_shape) ) for i in X_series])

In [8]:
# Encode pitches

le = LabelEncoder() 
y = le.fit_transform(y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Reshape for model
# X_train = np.array([X.reshape(20, 20, 1) for x in X_train])
# X_test = np.array([X.reshape(20, 20, 1) for x in X_test])

# onehotencoder = OneHotEncoder() 
y_test_hot = to_categorical(y_test)
y_train_hot = to_categorical(y_train)

In [9]:
# Troubleshooting queries
type(X_train[1])
X_train[1].shape
# X_train[1]

(22, 128, 1)

In [10]:
# # Model
# model = Sequential()
# # input_shape=(128, 128, 1)

# model.add(Conv2D(24, (5, 5), strides=(1, 1), input_shape=spectrogram_shape))
# model.add(MaxPooling2D((4, 2), strides=(4, 2)))
# model.add(Activation('relu'))

# model.add(Conv2D(48, (5, 5), padding="valid"))
# model.add(MaxPooling2D((4, 2), strides=(4, 2)))
# model.add(Activation('relu'))

# model.add(Conv2D(48, (5, 5), padding="valid"))
# model.add(Activation('relu'))

# model.add(Flatten())
# model.add(Dropout(rate=0.5))

# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dropout(rate=0.5))

# model.add(Dense(10))
# model.add(Activation('softmax'))

In [11]:
# model.compile(
# 	optimizer="Adam",
# 	loss="categorical_crossentropy",
# 	metrics=['accuracy'])

# model.fit(
# 	x=X_train, 
# 	y=y_train,
#     epochs=12,
#     batch_size=batch,
#     validation_data= (X_test, y_test))

# score = model.evaluate(
# 	x=X_test,
# 	y=y_test)

# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

In [12]:
# Model
fashion_model = Sequential()
fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(spectrogram_shape),padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.1))                  
fashion_model.add(Dense(12, activation='softmax'))

In [13]:
fashion_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [14]:
# fashion_model.summary()

In [15]:
fashion_train = fashion_model.fit(X_train, y_train_hot, batch_size=batch,epochs=8,verbose=1,validation_data=(X_test, y_test_hot))

Train on 2184 samples, validate on 729 samples
Epoch 1/8
2184/2184 [==============================] - 9s 4ms/step - loss: 2.4100 - accuracy: 0.2271 - val_loss: 1.9623 - val_accuracy: 0.4156
Epoch 2/8
2184/2184 [==============================] - 7s 3ms/step - loss: 1.0749 - accuracy: 0.7005 - val_loss: 0.5134 - val_accuracy: 0.8368
Epoch 3/8
2184/2184 [==============================] - 7s 3ms/step - loss: 0.4148 - accuracy: 0.8777 - val_loss: 0.3784 - val_accuracy: 0.8793
Epoch 4/8
2184/2184 [==============================] - 7s 3ms/step - loss: 0.2704 - accuracy: 0.9162 - val_loss: 0.3413 - val_accuracy: 0.8985
Epoch 5/8
2184/2184 [==============================] - 7s 3ms/step - loss: 0.1830 - accuracy: 0.9414 - val_loss: 0.3042 - val_accuracy: 0.9067
Epoch 6/8
2184/2184 [==============================] - 8s 4ms/step - loss: 0.1179 - accuracy: 0.9652 - val_loss: 0.2864 - val_accuracy: 0.9204
Epoch 7/8
2184/2184 [==============================] - 8s 4ms/step - loss: 0.0631 - accuracy: 0